In [1]:
import tensorflow as tf
import numpy as np

In [200]:
# 4 dimensions for each word
import math
def positionalEncodings(sequence):
    
    PE=[]
    d_model=4
    for pos in range(sequence):
        EncodingsOfEach=[]
        for i in range(d_model):
            if i%2==0:
                temp=pos/(math.pow(10000,(2*i)/d_model))
                EncodingsOfEach.append(math.sin(temp))
            else:
                temp=pos/(math.pow(10000,(2*(i-1))/d_model))
                EncodingsOfEach.append(math.cos(temp))
        PE.append(EncodingsOfEach)
    return PE

In [217]:
# def selfAttention(embeddings, W_q, W_k, W_v):
#     batch_size = tf.shape(embeddings)[0]
#     seq_len = tf.shape(embeddings)[1]

#     batch_outputs = []

#     for b in range(batch_size):
#         x = embeddings[b]            # (seq_len, d_model)

#         Q, K, V = [], [], []
#         for i in range(seq_len):
#             x_i = tf.expand_dims(x[i], 0)   # (1, d_model)
#             Q.append(tf.matmul(x_i, W_q))
#             K.append(tf.matmul(x_i, W_k))
#             V.append(tf.matmul(x_i, W_v))

#         d_k = tf.math.sqrt(tf.cast(tf.shape(K[0])[1], tf.float32))

#         attention_outputs = []

#         for i in range(seq_len):
#             score_list = []
#             for j in range(seq_len):
#                 score = tf.matmul(Q[i], K[j], transpose_b=True)[0, 0] / d_k
#                 score_list.append(score)

#             scores = tf.stack(score_list)
#             weights = tf.nn.softmax(scores)

#             output = tf.zeros_like(V[0])
#             for j in range(seq_len):
#                 output += tf.expand_dims(weights[j], axis=0) * V[j]

#             attention_outputs.append(output[0])

#         batch_outputs.append(tf.stack(attention_outputs))  # (seq_len, d_model)

#     return tf.stack(batch_outputs)  # (batch_size, seq_len, d_model)

def selfAttention(x, W_q, W_k, W_v):
    """
    x: (batch_size, seq_len, d_model)
    W_q, W_k, W_v: (d_model, d_model)
    """

    # Linear projections
    Q = tf.matmul(x, W_q)  # (batch, seq_len, d_model)
    K = tf.matmul(x, W_k)
    V = tf.matmul(x, W_v)

    d_k = tf.cast(tf.shape(K)[-1], tf.float32)

    # Scaled dot-product attention
    scores = tf.matmul(Q, K, transpose_b=True)  # (batch, seq_len, seq_len)
    scores = scores / tf.math.sqrt(d_k)

    weights = tf.nn.softmax(scores, axis=-1)    # (batch, seq_len, seq_len)

    output = tf.matmul(weights, V)               # (batch, seq_len, d_model)

    return output


In [202]:
def Add_and_normalize(output_of_previous,input_of_previous):
    layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    return layer_norm(output_of_previous + input_of_previous)

In [203]:
def FeedForwardNN(x, W1, b1, W2, b2):
    hidden = tf.nn.relu(tf.matmul(x, W1) + b1)
    out = tf.matmul(hidden, W2) + b2
    return out


In [204]:
def Encoder(positional_encodings_plus_embeddings,W_q,W_k,W_v,W1,b1,W2,b2):
    self_attn_output=selfAttention(positional_encodings_plus_embeddings,W_q,W_k,W_v)
    add_and_normalize_1_output=Add_and_normalize(self_attn_output,positional_encodings_plus_embeddings)
    ffn_output=FeedForwardNN(add_and_normalize_1_output,W1,b1,W2,b2)
    encoder_output=Add_and_normalize(ffn_output,add_and_normalize_1_output)
    return encoder_output

In [7]:
embeddings=tf.constant([[1,0,1,0],[0,1,0,1],[1,1,1,1]],dtype=tf.float32)
positional_encodings=positionalEncodings(len(embeddings))
embeddings_plus_PE=embeddings+positional_encodings
d_model=embeddings_plus_PE[0].shape[0]
encoder_params = []
for _ in range(6):
    params = {
        "W_q": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W1": tf.Variable(tf.random.normal(shape=(d_model,256))),
        "b1": tf.Variable(tf.random.normal(shape=(256,))),
        "W2": tf.Variable(tf.random.normal(shape=(256,d_model))),
        "b2": tf.Variable(tf.random.normal(shape=(d_model,)))
    }
    encoder_params.append(params)


In [198]:
def encoderStack(x, encoder_params):
    for layer in encoder_params:
        x = Encoder(
            x,
            layer["W_q"],
            layer["W_k"],
            layer["W_v"],
            layer["W1"],
            layer["b1"],
            layer["W2"],
            layer["b2"]
        )
    return x


In [9]:
encoder_output=(encoderStack(embeddings_plus_PE,encoder_params))

In [10]:
encoder_output

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-1.3913472 ,  0.6703489 , -0.46256405,  1.1835624 ],
       [-1.3935652 ,  0.6702541 , -0.45905954,  1.1823704 ],
       [-1.3927484 ,  0.67031825, -0.46036035,  1.1827906 ]],
      dtype=float32)>

Decoder:


In [11]:
def masked_Attention(embeddings, W_q, W_k, W_v,mask):
    numberOfWords = tf.shape(embeddings)[0]

    Q, K, V = [], [], []
    for i in range(numberOfWords):
        x_i = tf.expand_dims(embeddings[i], 0)
        Q.append(tf.matmul(x_i, W_q))
        K.append(tf.matmul(x_i, W_k))
        V.append(tf.matmul(x_i, W_v))

    d_k = tf.math.sqrt(tf.cast(tf.shape(K[0])[1], tf.float32))

    attention_outputs = []

    for i in range(numberOfWords):
        score_list = []
        for j in range(numberOfWords):
            score = tf.matmul(Q[i], K[j], transpose_b=True)[0, 0] / d_k
            score=score+mask[i][j]
            score_list.append(score)

        scores = tf.stack(score_list)
        weights = tf.nn.softmax(scores)

        output = tf.zeros_like(V[0])
        for j in range(numberOfWords):
            output += weights[j] * V[j]

        attention_outputs.append(output[0])

    return tf.stack(attention_outputs)


In [206]:
def Encoder_decoder_attention(encoder_output,prev_layer_output,W_q,W_k,W_v):
    numberOfWords = tf.shape(encoder_output)[0]

    Q, K, V = [], [], []
    for i in range(numberOfWords):
        q_i = tf.expand_dims(prev_layer_output[i], 0)
        enc_i = tf.expand_dims(encoder_output[i], 0)
        Q.append(tf.matmul(q_i, W_q))
        K.append(tf.matmul(enc_i, W_k))
        V.append(tf.matmul(enc_i, W_v))

    d_k = tf.math.sqrt(tf.cast(tf.shape(K[0])[1], tf.float32))

    attention_outputs = []

    for i in range(numberOfWords):
        score_list = []
        for j in range(numberOfWords):
            score = tf.matmul(Q[i], K[j], transpose_b=True)[0, 0] / d_k
            score_list.append(score)

        scores = tf.stack(score_list)
        weights = tf.nn.softmax(scores)

        output = tf.zeros_like(V[0])
        for j in range(numberOfWords):
            output += tf.expand_dims(weights[j], axis=0) * V[j]

        attention_outputs.append(output[0])

    return tf.stack(attention_outputs)
    

In [13]:
def decoder(positional_encodings_plus_embeddings,encoder_output,W_q_self,W_k_self,W_v_self,W_q_cross,W_k_cross,W_v_cross,mask,W1,b1,W2,b2):
    masked_self_attn_output=masked_Attention(positional_encodings_plus_embeddings,W_q_self,W_k_self,W_v_self,mask)
    add_and_normalize_1_output=Add_and_normalize(masked_self_attn_output,positional_encodings_plus_embeddings)
    encoder_decoder_attn_output=Encoder_decoder_attention(encoder_output,add_and_normalize_1_output,W_q_cross,W_k_cross,W_v_cross)
    add_and_normalize_2_output=Add_and_normalize(encoder_decoder_attn_output,add_and_normalize_1_output)
    ffn_output=FeedForwardNN(add_and_normalize_2_output,W1,b1,W2,b2)
    decoder_output=Add_and_normalize(ffn_output,add_and_normalize_2_output)
    return decoder_output

In [14]:
def causal_mask(d):
    mask = tf.linalg.band_part(tf.ones((d, d)), -1, 0)
    return (1.0 - mask) * -1e9


In [15]:
d_model=embeddings_plus_PE[0].shape[0]
mask=causal_mask(d_model)
decoder_params = []

for _ in range(6):
    params = {
        "W_q_self": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k_self": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v_self": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_q_cross": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k_cross": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v_cross": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W1": tf.Variable(tf.random.normal(shape=(d_model,256))),
        "b1": tf.Variable(tf.random.normal(shape=(256,))),
        "W2": tf.Variable(tf.random.normal(shape=(256,d_model))),
        "b2": tf.Variable(tf.random.normal(shape=(d_model,)))
    }
    decoder_params.append(params)

In [16]:
def decoderStack(x,decoder_params,encoder_output):
    for layer in decoder_params:
        x=decoder(x,encoder_output,layer["W_q_self"],layer["W_k_self"],layer["W_v_self"],layer["W_q_cross"],layer["W_k_cross"],layer["W_v_cross"],mask,layer["W1"],layer["b1"],layer["W2"],layer["b2"])
    return x

In [17]:
embeddings=tf.constant([[1,0,1,0],[0,1,1,0],[1,1,0,1]],dtype=tf.float32)
positional_encodings=positionalEncodings(len(embeddings))
embeddings_plus_PE=embeddings+positional_encodings

In [18]:
print(decoderStack(embeddings_plus_PE,decoder_params,encoder_output))

tf.Tensor(
[[ 0.83228827 -1.6974741   0.27196252  0.59322333]
 [ 0.8322718  -1.6974778   0.2719764   0.59322953]
 [ 0.8322394  -1.6974734   0.27192146  0.59331256]], shape=(3, 4), dtype=float32)


Preparing the dataset:


In [78]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [79]:
df=pd.read_csv('chatbot_dataset.csv')
df.head(1)

,User ID,User Utterance,Bot Response,Timestamp,Context/Session ID,Entities,User Feedback,Conversation Outcome,User Profile,Channel/Platform,Language,User Emotion/Sentiment,Location,User Segment
0,uKqYhMMQ5S,Charge bar between follow student.,Important law into large example range. Player...,2023-10-31 18:02:06,4dfe56dc-efe2-49c6-be9f-ce5b84ca4de4,event,negative,incomplete,Annette Henderson,social media,German,confused,Sydney,returning customers


In [108]:
x=df['User Utterance']
y=df['Bot Response']

In [109]:
x

0            Charge bar between follow student.
1               Bad every reflect huge contain.
2       Glass remember many dog director under.
3           Help charge record many talk tough.
4               Position not man much material.
                         ...                   
9995           How major might home gun reduce.
9996          Result executive some commercial.
9997                Require week affect factor.
9998                    Film I green like song.
9999                   Wonder majority million.
Name: User Utterance, Length: 10000, dtype: object

In [110]:
def clean_sentence(temp):
    temp=temp.lower()
    temp=temp.replace(".","")
    return temp

In [111]:
x=x.apply(clean_sentence)


In [112]:
x

0            charge bar between follow student
1               bad every reflect huge contain
2       glass remember many dog director under
3           help charge record many talk tough
4               position not man much material
                         ...                  
9995           how major might home gun reduce
9996          result executive some commercial
9997                require week affect factor
9998                    film i green like song
9999                   wonder majority million
Name: User Utterance, Length: 10000, dtype: object

In [113]:
y=y.apply(clean_sentence)

In [114]:
y_input="<sos> "+y
y_target=y+" <eos>"

In [115]:
from keras.preprocessing.text import Tokenizer

In [116]:
tokenizer=Tokenizer(filters='',oov_token="OOV")

In [117]:
tokenizer.fit_on_texts(list(x)+list(y_input)+list(y_target))


In [124]:
len(tokenizer.word_index)

1500

In [118]:
x=tokenizer.texts_to_sequences(x)


In [119]:
y_input=tokenizer.texts_to_sequences(y_input)
y_target=tokenizer.texts_to_sequences(y_target)

In [120]:
from tensorflow.keras.utils import pad_sequences
x=pad_sequences(x,padding='post')
y_input=pad_sequences(y_input,padding='post')
y_target=pad_sequences(y_target,padding='post')

In [121]:
numberOfWords=len(tokenizer.word_index)+1

In [128]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=numberOfWords,
    output_dim=64
)


In [135]:
x=embedding_layer(x)

In [136]:
y_input=embedding_layer(y_input)

In [140]:
y_target=embedding_layer(y_target)

In [176]:
# 4 dimensions for each word
import math
def positionalEncodings(sequence):
    
    PE=[]
    d_model=64
    for pos in range(sequence):
        EncodingsOfEach=[]
        for i in range(d_model):
            if i%2==0:
                temp=pos/(math.pow(10000,(2*i)/d_model))
                EncodingsOfEach.append(math.sin(temp))
            else:
                temp=pos/(math.pow(10000,(2*(i-1))/d_model))
                EncodingsOfEach.append(math.cos(temp))
        PE.append(EncodingsOfEach)
    return PE

In [183]:
PE=positionalEncodings(len(x[0]))

In [189]:
x=x+PE

In [192]:
type(x)

tensorflow.python.framework.ops.EagerTensor

In [214]:
x.shape

TensorShape([10000, 10, 64])

In [212]:

d_model=x[0].shape[1]
encoder_params = []
for _ in range(6):
    params = {
        "W_q": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W1": tf.Variable(tf.random.normal(shape=(d_model,256))),
        "b1": tf.Variable(tf.random.normal(shape=(256,))),
        "W2": tf.Variable(tf.random.normal(shape=(256,d_model))),
        "b2": tf.Variable(tf.random.normal(shape=(d_model,)))
    }
    encoder_params.append(params)


In [218]:
print(encoderStack(x,encoder_params))

tf.Tensor(
[[[-3.0989262e-01  1.4312212e+00 -1.1284838e+00 ... -2.3945420e+00
    2.2073835e-03  1.1830878e+00]
  [-3.0989212e-01  1.4312230e+00 -1.1284851e+00 ... -2.3945439e+00
    2.2078454e-03  1.1830875e+00]
  [-3.0989325e-01  1.4312245e+00 -1.1284866e+00 ... -2.3945451e+00
    2.2089034e-03  1.1830881e+00]
  ...
  [-3.0989361e-01  1.4312220e+00 -1.1284819e+00 ... -2.3945410e+00
    2.2056103e-03  1.1830901e+00]
  [-3.0989277e-01  1.4312241e+00 -1.1284810e+00 ... -2.3945405e+00
    2.2068322e-03  1.1830887e+00]
  [-3.0989277e-01  1.4312255e+00 -1.1284813e+00 ... -2.3945410e+00
    2.2076964e-03  1.1830879e+00]]

 [[-3.5908791e-01  1.3894553e+00 -1.0837812e+00 ... -2.4052894e+00
    1.3856903e-02  1.2016587e+00]
  [-3.5908574e-01  1.3894622e+00 -1.0837830e+00 ... -2.4052908e+00
    1.3856605e-02  1.2016572e+00]
  [-3.5908735e-01  1.3894629e+00 -1.0837842e+00 ... -2.4052920e+00
    1.3856620e-02  1.2016555e+00]
  ...
  [-3.5908723e-01  1.3894600e+00 -1.0837811e+00 ... -2.4052896e+00